# Predicting Baseball Salaries, Part II

I previously worked on a simple regression model to predict baseball player salaries where I collected player team, position and salary for the 2019 season. It was a simple exercise in using decision trees and different ensemble methods to predict salaries, but I wanted to go back adn see if I included more stats, specifically the previous season's hitting statistics, could I get closer to predicting salary?

My first step was to load in the libraries necessary and then my previously collected salary data from [USA Today](https://www.usatoday.com/sports/mlb/salaries/).

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
salary = pd.read_pickle('salary_info')

In [4]:
salary.head()

,name,team,position,salary
0,MaxScherzer,WSH,SP,42142857
1,StephenStrasburg,WSH,SP,36428571
2,MikeTrout,LAA,CF,34083333
3,ZackGreinke,ARI,SP,32421884
4,DavidPrice,BOS,SP,31000000


Now that I have those salaraies, I want to get the 2018 hitting stats. I'm going to collect my data from [MLB's site](https://www.mlb.com/stats/2018) for each player and then use BeautifulSoup to parse the HTML.

In [5]:
url = 'https://www.mlb.com/stats/2018'
html = requests.get(url)
html

<Response [200]>

In [6]:
soup = BeautifulSoup(html.content, 'html.parser')

In [126]:
name_container = soup.find('th', {'data-row' : 0})
name_container

<th class="pinned-col-3lxtuFnc col-group-start-sa9unvY0 number-aY5arzrB first-col-3aGPCzvr is-table-pinned-1WfPW2jT" data-col="0" data-row="0" id="tb-1913-body-row0" scope="row"><div class="custom-cell-wrapper-34Cjf9P0"><div class="index-3cdMSKi7">1</div><div class="value-wrapper-1W5GYs5E"><div class="top-wrapper-1NLTqKbE"><div><a aria-label="Mike Trout" class="bui-link" href="/player/545361"><span class="full-3fV3c9pF">Mike</span><span class="short-3OJ0bTju">M Trout</span><span class="full-3fV3c9pF">Trout</span></a></div><div class="position-28TbwVOg">CF</div></div></div></div><div class="placeholder-wrapper-bEG1UFFP"><div class="index-3cdMSKi7">1</div><div><span class="bui-skeleton"><span class="skeleton-row-2cL12jX9" style="background-color:#eee;background-image:linear-gradient(90deg, #eee, #F5F5F5, #eee);border-radius:50%;width:42px;height:42px">‌</span></span></div><div class="placeholder-content-2l2UMerJ"><div><span class="bui-skeleton"><span class="skeleton-row-2cL12jX9" style="

In [128]:
names = name_container.findAll('span', class_ = 'full-3fV3c9pF')
names

[<span class="full-3fV3c9pF">Mike</span>,
 <span class="full-3fV3c9pF">Trout</span>]

In [75]:
for i in range(len(names)):
    print(names[i].get_text())

Mike
Trout


Get Name

In [82]:
first = names[0].get_text()
last = names[1].get_text()
print(first, last)

Mike Trout


In [131]:
full_name = first + " " + last
full_name

'Mike Trout'

Get Position

In [93]:
name_container.find('div', class_ = 'position-28TbwVOg').get_text()

'CF'

Get Team

In [105]:
soup.find('td', {'data-col': '1', 'data-row': '0'}).get_text()

'LAA'

Get Games

In [106]:
soup.find('td', {'data-col': '2', 'data-row': '0'}).get_text()

'140'

Get At Bats

In [107]:
soup.find('td', {'data-col': '3', 'data-row': '0'}).get_text()

'471'

Get Runs

In [109]:
soup.find('td', {'data-col': '4', 'data-row': '0'}).get_text()

'101'

Get Hits

In [111]:
soup.find('td', {'data-col': '5', 'data-row': '0'}).get_text()

'147'

Get Doubles

In [113]:
soup.find('td', {'data-col': '6', 'data-row': '0'}).get_text()

'24'

Get Triples

In [114]:
soup.find('td', {'data-col': '7', 'data-row': '0'}).get_text()

'4'

Get Homeruns

In [115]:
soup.find('td', {'data-col': '8', 'data-row': '0'}).get_text()

'39'

Get RBIs

In [116]:
soup.find('td', {'data-col': '9', 'data-row': '0'}).get_text()

'79'

Get Walks

In [117]:
soup.find('td', {'data-col': '10', 'data-row': '0'}).get_text()

'122'

Get Strikeouts

In [118]:
soup.find('td', {'data-col': '11', 'data-row': '0'}).get_text()

'124'

Get Stolen Bases

In [119]:
soup.find('td', {'data-col': '12', 'data-row': '0'}).get_text()

'24'

Get Caught Stealing

In [120]:
soup.find('td', {'data-col': '13', 'data-row': '0'}).get_text()

'2'

Get Batting Average

In [121]:
soup.find('td', {'data-col': '14', 'data-row': '0'}).get_text()

'.312'

Get On Base Percentage

In [122]:
soup.find('td', {'data-col': '15', 'data-row': '0'}).get_text()

'.460'

Get Slugging Percentage

In [123]:
soup.find('td', {'data-col': '16', 'data-row': '0'}).get_text()

'.628'

Get On Base Plus Slugging Percentage

In [124]:
soup.find('td', {'data-col': '17', 'data-row': '0'}).get_text()

'1.088'

In [190]:
df = pd.DataFrame(columns = ['FirstName', 'LastName', 'FullName', 'Position', 'Team', 'Games', 'At_Bats', 'Runs', 
                             'Hits', 'Doubles', 'Triples', 'Homeruns', 'RBIs', 'Walks', 'Strikeouts', 'StolenBases', 
                            'CaughtStealing', 'BattingAverage', 'OnBasePercentage', 'SluggingPercentage', 
                            'OnBaseSluggingPercent'])
df

,FirstName,LastName,FullName,Position,Team,Games,At_Bats,Runs,Hits,Doubles,Triples,Homeruns,RBIs,Walks,Strikeouts,StolenBases,CaughtStealing,BattingAverage,OnBasePercentage,SluggingPercentage,OnBaseSluggingPercent


In [132]:
def get_first_name(data_row):
    soup.find('th', {'data-row' : data_row})
    names = container.findAll('span', class_ = 'full-3fV3c9pF')
    first_name = names[0].get_text()
    last_name = names[1].get_text()
    full_name = first_name + ' ' + last_name
    return first_name, last_name, full_name

In [134]:
def get_position(data_row):
    container = soup.find('th', {'data-row' : data_row})
    position = container.find('div', class_ = 'position-28TbwVOg').get_text()
    return position

In [137]:
def get_team(data_row):
    team = soup.find('td', {'data-col': '1', 'data-row': data_row}).get_text()
    return team

In [139]:
def get_games(data_row):
    games = soup.find('td', {'data-col': '2', 'data-row': data_row}).get_text()
    return games

In [141]:
def get_atbats(data_row):
    at_bats = soup.find('td', {'data-col': '3', 'data-row': data_row}).get_text()
    return at_bats

In [143]:
def get_runs(data_row):
    runs = soup.find('td', {'data-col': '4', 'data-row': data_row}).get_text()
    return runs

In [145]:
def get_hits(data_row):
    hits = soup.find('td', {'data-col': '5', 'data-row': data_row}).get_text()
    return hits

In [148]:
def get_doubles(data_row):
    doubles = soup.find('td', {'data-col': '6', 'data-row': data_row}).get_text()
    return doubles

In [150]:
def get_triples(data_row):
    triples = soup.find('td', {'data-col': '7', 'data-row': data_row}).get_text()
    return triples

In [152]:
def get_homeruns(data_row):
    homeruns = soup.find('td', {'data-col': '8', 'data-row': data_row}).get_text()
    return homeruns

In [154]:
def get_rbis(data_row):
    rbis = soup.find('td', {'data-col': '9', 'data-row': data_row}).get_text()
    return rbis

In [156]:
def get_walks(data_row):
    walks = soup.find('td', {'data-col': '10', 'data-row': data_row}).get_text()
    return walks

In [158]:
def get_strikeouts(data_row):
    strikeouts = soup.find('td', {'data-col': '11', 'data-row': data_row}).get_text()
    return strikeouts

In [160]:
def get_stolenbases(data_row):
    stolenbases = soup.find('td', {'data-col': '12', 'data-row': data_row}).get_text()
    return stolenbases

In [162]:
def get_caughtstealing(data_row):
    caughtstealing = soup.find('td', {'data-col': '13', 'data-row': data_row}).get_text()
    return caughtstealing

In [164]:
def get_battingavg(data_row):
    battingavg = soup.find('td', {'data-col': '14', 'data-row': data_row}).get_text()
    return battingavg

In [166]:
def get_onbasepercent(data_row):
    onbasepercent = soup.find('td', {'data-col': '15', 'data-row': data_row}).get_text()
    return onbasepercent

In [167]:
def get_sluggingpercent(data_row):
    sluggingpercent = soup.find('td', {'data-col': '16', 'data-row': data_row}).get_text()
    return sluggingpercent

In [168]:
def get_onbaseslugging(data_row):
    onbaseslugging = soup.find('td', {'data-col': '17', 'data-row': data_row}).get_text()
    return onbaseslugging

In [201]:
def get_page(webpage):
    html = requests.get(webpage)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

In [202]:
players_on_page = range(25)

def get_contents(webpage):
    soup = get_page(webpage)
    
    for i in players_on_page:
        first_name, last_name, full_name = get_first_name(i)
        position = get_position(i)
        team = get_team(i)
        games = get_games(i)
        at_bats = get_atbats(i)
        runs = get_runs(i)
        hits = get_hits(i)
        doubles = get_doubles(i)
        triples = get_triples(i)
        homeruns = get_homeruns(i)
        rbis = get_rbis(i)
        walks = get_walks(i)
        strikeouts = get_strikeouts(i)
        stolenbases = get_stolenbases(i)
        caughtstealing = get_caughtstealing(i)
        battingavg = get_battingavg(i)
        onbasepercent = get_onbasepercent(i)
        slugging = get_sluggingpercent(i)
        onbaseslug = get_onbaseslugging(i)
        
        global df
        df = df.append({'FirstName': first_name, 
                       'LastName': last_name,
                       'FullName': full_name, 
                       'Position': position, 
                       'Team': team,
                       'Games': games,
                       'At_Bats': at_bats,
                       'Runs': runs,
                       'Hits': hits,
                       'Doubles': doubles,
                       'Triples': triples,
                       'Homeruns': homeruns,
                       'RBIs': rbis,
                       'Walks': walks,
                       'Strikeouts': strikeouts,
                       'StolenBases': stolenbases,
                       'CaughStealing': caughtstealing,
                       'BattingAverage': battingavg,
                       'OnBasePercentage': onbasepercent,
                       'SluggingPercentage': slugging,
                       'OnBaseSluggingPercent': onbaseslug}, ignore_index=True)

In [191]:
first_url = 'https://www.mlb.com/stats/2018?playerPool=ALL'

In [192]:
get_contents(first_url)

In [193]:
df

,FirstName,LastName,FullName,Position,Team,Games,At_Bats,Runs,Hits,Doubles,Triples,Homeruns,RBIs,Walks,Strikeouts,StolenBases,CaughtStealing,BattingAverage,OnBasePercentage,SluggingPercentage,OnBaseSluggingPercent,CaughStealing
0,Mike,Trout,Mike Trout,CF,LAA,140,471,101,147,24,4,39,79,122,124,24,NaN,.312,.460,.628,1.088,2
1,Mookie,Betts,Mookie Betts,RF,BOS,136,520,129,180,47,5,32,80,81,91,30,NaN,.346,.438,.640,1.078,6
2,J.D.,Martinez,J.D. Martinez,LF,BOS,150,569,111,188,37,2,43,130,69,146,6,NaN,.330,.402,.629,1.031,1
3,Christian,Yelich,Christian Yelich,RF,MIL,147,574,118,187,34,7,36,110,68,135,22,NaN,.326,.402,.598,1.000,4
4,Jose,Ramirez,Jose Ramirez,3B,CLE,157,578,110,156,38,4,39,105,106,80,34,NaN,.270,.387,.552,.939,6
5,Nolan,Arenado,Nolan Arenado,3B,COL,156,590,104,175,38,2,38,110,73,122,2,NaN,.297,.374,.561,.934,2
6,Alex,Bregman,Alex Bregman,3B,HOU,157,594,105,170,51,1,31,103,96,85,10,NaN,.286,.394,.532,.926,4
7,Paul,Goldschmidt,Paul Goldschmidt,1B,ARI,158,593,95,172,35,5,33,83,90,173,7,NaN,.290,.389,.533,.922,4
8,Trevor,Story,Trevor Story,SS,COL,157,598,88,174,42,6,37,108,47,168,27,NaN,.291,.348,.567,.914,6
9,Anthony,Rendon,Anthony Rendon,3B,WSH,136,529,88,163,44,2,24,92,55,82,2,NaN,.308,.374,.535,.908,1


In [194]:
url_list = []

for i in range(2, 52):
    url_list.append('https://www.mlb.com/stats/2018?page=' + str(i) + '&playerPool=ALL')

url_list

['https://www.mlb.com/stats/2018?page=2&playerPool=ALL',
 'https://www.mlb.com/stats/2018?page=3&playerPool=ALL',
 'https://www.mlb.com/stats/2018?page=4&playerPool=ALL',
 'https://www.mlb.com/stats/2018?page=5&playerPool=ALL',
 'https://www.mlb.com/stats/2018?page=6&playerPool=ALL',
 'https://www.mlb.com/stats/2018?page=7&playerPool=ALL',
 'https://www.mlb.com/stats/2018?page=8&playerPool=ALL',
 'https://www.mlb.com/stats/2018?page=9&playerPool=ALL',
 'https://www.mlb.com/stats/2018?page=10&playerPool=ALL',
 'https://www.mlb.com/stats/2018?page=11&playerPool=ALL',
 'https://www.mlb.com/stats/2018?page=12&playerPool=ALL',
 'https://www.mlb.com/stats/2018?page=13&playerPool=ALL',
 'https://www.mlb.com/stats/2018?page=14&playerPool=ALL',
 'https://www.mlb.com/stats/2018?page=15&playerPool=ALL',
 'https://www.mlb.com/stats/2018?page=16&playerPool=ALL',
 'https://www.mlb.com/stats/2018?page=17&playerPool=ALL',
 'https://www.mlb.com/stats/2018?page=18&playerPool=ALL',
 'https://www.mlb.com/

In [200]:
for i in url_list:
    print(i)
#    get_contents(i)

https://www.mlb.com/stats/2018?page=2&playerPool=ALL
https://www.mlb.com/stats/2018?page=3&playerPool=ALL
https://www.mlb.com/stats/2018?page=4&playerPool=ALL
https://www.mlb.com/stats/2018?page=5&playerPool=ALL
https://www.mlb.com/stats/2018?page=6&playerPool=ALL
https://www.mlb.com/stats/2018?page=7&playerPool=ALL
https://www.mlb.com/stats/2018?page=8&playerPool=ALL
https://www.mlb.com/stats/2018?page=9&playerPool=ALL
https://www.mlb.com/stats/2018?page=10&playerPool=ALL
https://www.mlb.com/stats/2018?page=11&playerPool=ALL
https://www.mlb.com/stats/2018?page=12&playerPool=ALL
https://www.mlb.com/stats/2018?page=13&playerPool=ALL
https://www.mlb.com/stats/2018?page=14&playerPool=ALL
https://www.mlb.com/stats/2018?page=15&playerPool=ALL
https://www.mlb.com/stats/2018?page=16&playerPool=ALL
https://www.mlb.com/stats/2018?page=17&playerPool=ALL
https://www.mlb.com/stats/2018?page=18&playerPool=ALL
https://www.mlb.com/stats/2018?page=19&playerPool=ALL
https://www.mlb.com/stats/2018?page=

In [196]:
df.shape

(1275, 22)

In [197]:
salary.shape

(877, 4)

In [198]:
df.FullName.value_counts()

Anthony Rendon      51
Khris Davis         51
David Peralta       51
Francisco Lindor    51
Christian Yelich    51
Matt Chapman        51
Javier Baez         51
Charlie Blackmon    51
Manny Machado       51
Mookie Betts        51
Trevor Story        51
Jesus Aguilar       51
Paul Goldschmidt    51
Bryce Harper        51
Mitch Haniger       51
Matt Carpenter      51
Freddie Freeman     51
J.D. Martinez       51
Mike Trout          51
Eugenio Suarez      51
Brandon Nimmo       51
Alex Bregman        51
Nolan Arenado       51
Xander Bogaerts     51
Jose Ramirez        51
Name: FullName, dtype: int64